In [14]:
# ==============================================================================
# 1. 라이브러리 설치 및 데이터 다운로드
# ==============================================================================

!pip install efficientnet_pytorch
!pip install torchmetrics
!pip install kagglehub  # KaggleHub 설치

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import kagglehub # 임포트
import time
import copy
from google.colab import drive
from collections import Counter # ⚠️ 클래스 카운트를 위한 라이브러리 추가
from google.colab import drive #

# 다운로드 및 경로 설정
path = kagglehub.dataset_download("kaustubhb999/tomatoleaf")

print("다운로드된 데이터셋 경로:", path)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


100%|██████████| 179M/179M [00:01<00:00, 165MB/s]

Extracting files...


다운로드된 데이터셋 경로: /root/.cache/kagglehub/datasets/kaustubhb999/tomatoleaf/versions/1
사용 장치: cuda:0


In [19]:
# ==============================================================================
# 2. 데이터셋 경로 설정
# ==============================================================================
DATA_ROOT = path

# 🚨 데이터셋 구조: kaustubhb999/tomatoleaf는 압축 해제 후 'tomato' 폴더를 생성합니다.
# NESTED_ROOT = DATA_ROOT (예: /root/.cache/.../kaustubhb999/tomatoleaf/versions/1) + 'tomato'
NESTED_ROOT = os.path.join(DATA_ROOT, 'tomato')

# 💡 최종 TRAIN과 VAL 경로 설정
TRAIN_DIR = os.path.join(NESTED_ROOT, 'train')
VAL_DIR = os.path.join(NESTED_ROOT, 'val')

print(f"✅ 최종 TRAIN 경로: {TRAIN_DIR}")
print(f"✅ 최종 VAL 경로: {VAL_DIR}")

✅ 최종 TRAIN 경로: /root/.cache/kagglehub/datasets/kaustubhb999/tomatoleaf/versions/1/tomato/train
✅ 최종 VAL 경로: /root/.cache/kagglehub/datasets/kaustubhb999/tomatoleaf/versions/1/tomato/val


In [ ]:
# ==============================================================================
# 3. 하이퍼파라미터 및 데이터 변환 설정 (모델 축소 적용)
# ==============================================================================
from efficientnet_pytorch import EfficientNet

# 하이퍼파라미터
BATCH_SIZE = 32 # B0 모델에 맞춰 배치 사이즈를 32로 늘려 속도 개선
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001 # 과적합 방지를 위해 학습률은 계속 낮게 유지
MODEL_NAME = 'efficientnet-b0' 

# EfficientNet-B0의 권장 입력 이미지 크기 (224x224로 자동 변경됨)
INPUT_SIZE = EfficientNet.get_image_size(MODEL_NAME)

# 데이터 증강 및 정규화
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.RandomResizedCrop(INPUT_SIZE),
        
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 통계
    ]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# ==============================================================================
# 4. 데이터 로드 및 DataLoader 생성 (수정 완료)
# ==============================================================================

print("\n데이터 로딩 중...")

image_datasets = {
    'train': datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    # 'val' 데이터셋을 로드하되, 변환은 'test' 키를 사용합니다.
    'val': datasets.ImageFolder(VAL_DIR, data_transforms['test']) # 👈 'val' 대신 'test' 사용
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
    'val': DataLoader(image_datasets['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
NUM_CLASSES = len(class_names)

# === train/val 라벨 매핑 강제 통일
image_datasets['val'].class_to_idx = image_datasets['train'].class_to_idx

print(f"클래스 개수: {NUM_CLASSES}")
print(f"클래스 이름: {class_names}")
print(f"학습 데이터셋 크기: {dataset_sizes['train']}")
print(f"검증 데이터셋 크기: {dataset_sizes['val']}")


데이터 로딩 중...
클래스 개수: 10
클래스 이름: ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']
학습 데이터셋 크기: 10000
검증 데이터셋 크기: 1000


In [ ]:
# ==============================================================================
# 5. 모델 설정 및 가중치 부여 손실 함수 설정 
# ==============================================================================
from collections import Counter
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# ----------------------------------------------------------------------
# 1. 클래스 가중치 계산 (Class Imbalance 대처)
# ----------------------------------------------------------------------

# ⚠️ image_datasets, NUM_CLASSES, device 변수가 섹션 4에서 정의되었다고 가정합니다.
# 이 코드를 실행하기 전에 섹션 4의 데이터 로딩 부분이 실행되어 image_datasets가 정의되어야 합니다.
train_targets = image_datasets['train'].targets
class_counts = Counter(train_targets)
total_count = sum(class_counts.values())

class_weights = [total_count / class_counts[i] for i in range(NUM_CLASSES)]
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights / class_weights.min()

print(f"✅ 클래스 가중치 계산 완료: {class_weights.tolist()}")


# ----------------------------------------------------------------------
# 2. EfficientNet-B0 모델 로드 및 분류 레이어 명시적 교체 
# ----------------------------------------------------------------------

# ImageNet 가중치로 EfficientNet-B0 모델을 로드합니다.
# MODEL_NAME 변수는 이제 'efficientnet-b0'입니다.
model_ft = EfficientNet.from_pretrained(MODEL_NAME).to(device)

# 기존 분류 레이어(_fc)의 입력 피처 수 확인
num_ftrs = model_ft._fc.in_features

# 💡 [수정] 실제 데이터셋의 클래스 수에 맞춰 새로운 Sequential 레이어 정의 및 교체
#      (Dropout(0.5)을 추가하여 과적합 대처)
model_ft._fc = nn.Sequential(
    nn.Dropout(0.5), # 50%의 확률로 Dropout 적용
    nn.Linear(num_ftrs, NUM_CLASSES)
)

model_ft = model_ft.to(device) # 장치에 다시 올립니다.

print(f"✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 {num_ftrs} -> {NUM_CLASSES}로 성공적으로 교체되었습니다.")


# ----------------------------------------------------------------------
# 3. Loss, Optimizer, Scheduler 정의
# ----------------------------------------------------------------------

# 계산된 class_weights를 GPU/CPU 장치에 올리고 CrossEntropyLoss에 적용
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# AdamW 옵티마이저 사용 (수정된 LEARNING_RATE 적용)
optimizer_ft = optim.AdamW(model_ft.parameters(), lr=LEARNING_RATE)

# 학습률 스케줄러: 8 에포크마다 0.5배 감소로 완화
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.5)

print("✅ Loss, Optimizer, Scheduler 정의 완료.")


✅ 클래스 가중치 계산 완료: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Loaded pretrained weights for efficientnet-b0
✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 1280 -> 10로 성공적으로 교체되었습니다.
✅ Loss, Optimizer, Scheduler 정의 완료.


In [ ]:
# ==============================================================================
# 6. 모델 학습 함수 
# ==============================================================================
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print(f'\nEpoch {epoch+1}/{num_epochs}'); print('-' * 10)

        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

           
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'-> New best model found! Saving weights with Acc: {best_acc:.4f}')

        epoch_time = time.time() - epoch_start_time
        print(f'Epoch Time: {epoch_time:.0f}초')

    time_elapsed = time.time() - since
    print(f'\n학습 완료! 총 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최고 검증 정확도: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [32]:
# ==============================================================================
# 7. 모델 학습 실행 및 저장
# ==============================================================================

# 모델 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

# Google Drive 마운트
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/disease_model.pth' # ⚠️ 파일 이름에 weighted 추가

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 최적 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")


Epoch 1/10
----------
train Loss: 1.2338 Acc: 0.6391
val Loss: 0.2806 Acc: 0.9190
-> New best model found! Saving weights with Acc: 0.9190
Epoch Time: 78초

Epoch 2/10
----------
train Loss: 0.3647 Acc: 0.8858
val Loss: 0.1561 Acc: 0.9570
-> New best model found! Saving weights with Acc: 0.9570
Epoch Time: 75초

Epoch 3/10
----------
train Loss: 0.2446 Acc: 0.9241
val Loss: 0.1108 Acc: 0.9660
-> New best model found! Saving weights with Acc: 0.9660
Epoch Time: 76초

Epoch 4/10
----------
train Loss: 0.2027 Acc: 0.9360
val Loss: 0.0947 Acc: 0.9700
-> New best model found! Saving weights with Acc: 0.9700
Epoch Time: 76초

Epoch 5/10
----------
train Loss: 0.1777 Acc: 0.9422
val Loss: 0.0900 Acc: 0.9680
Epoch Time: 76초

Epoch 6/10
----------
train Loss: 0.1567 Acc: 0.9491
val Loss: 0.0811 Acc: 0.9690
Epoch Time: 77초

Epoch 7/10
----------
train Loss: 0.1356 Acc: 0.9573
val Loss: 0.0677 Acc: 0.9780
-> New best model found! Saving weights with Acc: 0.9780
Epoch Time: 76초

Epoch 8/10
----------